NLP Practical 2 (faking data and equal representation)

In [1]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [2]:
import urllib.request
import zipfile
import lxml.etree

In [3]:
import tensorflow as tf

In [4]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
input_text = doc.xpath('//content/text()')
input_label = doc.xpath('//head/keywords/text()')

del doc

In [5]:
talk_sentences = []
talknum = len(input_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', input_text[i])
    temp = re.sub(r'\n', '', input_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [6]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [7]:
talk_sentences_token = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentences_token.append(temp)

In [8]:
talk_sentences_token[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [9]:
input_label_prepro = []

for i in range(len(input_label)):
    temp = re.sub(r' ', '', input_label[i])
    input_label_prepro.append(temp.split(','))
    
#print(input_label_prepro)

In [10]:
labels = []

for i in range(len(input_label_prepro)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label_prepro[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label_prepro[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label_prepro[i]):
        temp[2] = 1
    labels.append(temp)
        
print(input_label_prepro[9])
print(labels[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [11]:
# labels_onehot = np.zeros(len(input_label_prepro), 8)
# for i in range(len(input_label_prepro)):
#     labels_onehot[i][labels[i][0]*4+labels[i][1]*2+labels[i][2]*1] = 1

In [12]:
labels_onehot = []
for i in range(len(input_label_prepro)):
    temp = np.zeros(8)
    hot = labels[i][0]*4+labels[i][1]*2+labels[i][2]*1
    temp[int(hot)] = 1
    labels_onehot.append(temp)

In [13]:
labels_onehot[:10]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])]

In [14]:
train_data = talk_sentences_token[:1835]
test_data = talk_sentences_token[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [15]:
doc = open("stopwords.txt", "r")
stopwords = doc.read().split('\n')

In [16]:
print(np.shape(talk_sentences_token))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 8)
(1835,)
(1835, 8)
(250,)
(250, 8)


In [17]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.57493188  0.0746594   0.0719346   0.00980926  0.17711172  0.06376022
  0.01525886  0.01253406]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]


In [18]:
train_sent = []

for i in range(len(train_data)):
    for sent in train_data[i]:
        train_sent.append(sent)
    
print(np.size(train_sent))

220963


In [19]:
from gensim.models import Word2Vec

C:\Users\lgligic\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [20]:
model_ted = Word2Vec(train_sent, min_count=1, size=100)

In [21]:
'hello' in model_ted.wv.vocab.keys()

True

In [22]:
'technology' in stopwords

False

In [23]:
c = 0
for talk in train_data:
    for sent in talk:
        for word in sent:
            c += 1
print(c / len(train_data))

c = 0
for talk in test_data:
    for sent in talk:
        for word in sent:
            c += 1
print(c / len(test_data))

2091.7547683923704
2615.152


In [24]:
sum(train_labels_onehot)

array([ 1055.,   137.,   132.,    18.,   325.,   117.,    28.,    23.])

In [25]:
removed = 1
while removed != 0:
    removed = 0
    t = 0
    for talk in train_data:
        s = 0
        for sent in talk:
            for word in sent:
                if word in stopwords:
                    train_data[t][s].remove(word)
                    removed += 1
            s += 1
        t += 1
    print(removed)

1439131
607421
125761
7876
87
0


In [26]:
removed = 1
while removed != 0:
    removed = 0
    t = 0
    for talk in test_data:
        s = 0
        for sent in talk:
            for word in sent:
                if word in stopwords:
                    test_data[t][s].remove(word)
                    removed += 1
            s += 1
        t += 1
    print(removed)

245657
104781
21908
1389
18
0


In [27]:
removed = 1
while removed != 0:
    removed = 0
    t = 0
    for talk in train_data:
        s = 0
        for sent in talk:
            if len(sent) == 0:
                train_data[t].remove(sent)
                removed += 1
            s += 1
        t += 1
    print(removed)

2114
561
153
41
5
0


In [28]:
removed = 1
while removed != 0:
    removed = 0
    t = 0
    for talk in test_data:
        s = 0
        for sent in talk:
            if len(sent) == 0:
                test_data[t].remove(sent)
                removed += 1
            s += 1
        t += 1
    print(removed)

550
179
58
15
5
1
0


In [29]:
c = 0
for talk in train_data:
    for sent in talk:
        for word in sent:
            c += 1
print(c / len(train_data))

c = 0
for talk in test_data:
    for sent in talk:
        for word in sent:
            c += 1
print(c / len(test_data))

903.5934604904633
1120.14


In [30]:
import random

train_data_expanded = train_data
train_labels_onehot_expanded = train_labels_onehot

for i in range(7):
    pool = np.array(train_data)[np.array(train_labels_onehot)[:,i+1] == 1]
    for j in range(int(sum(train_labels_onehot)[0] - sum(train_labels_onehot)[i+1])):
        temptext = []
        templabel = np.zeros(8)
        for k in range(1000):
            talk = random.choice(pool)
            sent = random.choice(talk)
            word = random.choice(sent)
            temptext.append(word)
        train_data_expanded.append(temptext)
        templabel[i+1] = 1
        train_labels_onehot_expanded.append(templabel)        

In [34]:
print(len(train_data_expanded))
print(len(train_labels_onehot_expanded))
print(sum(train_labels_onehot_expanded))

8440
8440
[ 1055.  1055.  1055.  1055.  1055.  1055.  1055.  1055.]


In [35]:
train_bow = []

for talk in train_data_expanded:
    bow = np.zeros(100)
    count = 0
    for sent in talk:
        for word in sent:
            bow += model_ted[word]
            count += 1
    bow = bow / count
    train_bow.append(bow)
    
print(len(train_bow))

8440


In [36]:
test_bow = []

for talk in test_data:
    bow = np.zeros(100)
    count = 0
    for sent in talk:
        for word in sent:
            if word in model_ted.wv.vocab.keys():
                bow += model_ted[word]
                count += 1
    bow = bow / count
    test_bow.append(bow)
    
print(len(test_bow))

250


In [37]:
class Iterator():
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.size = len(self.texts)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        temp = list(zip(self.texts, self.labels))
        random.shuffle(temp)
        self.texts, self.labels = zip(*temp)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res_texts = self.texts[self.cursor:self.cursor+n]
        res_labels = self.labels[self.cursor:self.cursor+n]
        self.cursor += n
        return res_texts, res_labels

In [38]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [46]:
node_count_1 = 100
node_count_2 = 100

x = tf.placeholder(tf.float32, shape=[None, 100])
y_ = tf.placeholder(tf.float32, shape=[None, 8])

# Define the first layer here
W1 = weight_variable([100, node_count_1])
b1 = bias_variable([node_count_1])
h1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

# Use dropout for this layer (should you wish)
keep_prob = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h1, keep_prob)

# Define the second layer here
W2 = weight_variable([node_count_1, node_count_2])
b2 = bias_variable([node_count_2])
h2 = tf.nn.sigmoid(tf.matmul(h_drop, W2) + b2)

# Define the output layer here
V = weight_variable([node_count_2, 8])
c = bias_variable([8])
y = tf.nn.softmax(tf.matmul(h2, V) + c)

# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cross_entropy)

# Start a tf session and run the optimisation algorithm
sess = tf.Session()
sess.run(tf.initialize_all_variables())

training = Iterator(train_bow, train_labels_onehot_expanded)
testing = Iterator(test_bow, test_labels_onehot)
train_accuracy = 0
test_accuracy = 0
N = 0

print(training.epochs)

while training.epochs < 10000:
    trd, trl = training.next_batch(50)
    if N % 10000 == 0:
        ted, tel = testing.next_batch(250)
        train_accuracy = sess.run(accuracy, feed_dict={x: trd, y_: trl, keep_prob: 1.0})
        test_accuracy = sess.run(accuracy, feed_dict={x: ted, y_: tel, keep_prob: 1.0})
        print("Epoch: %d, Training Accuracy: %f, Test Accuracy: %f" % (training.epochs, train_accuracy, test_accuracy))
    sess.run(train_step, feed_dict={x: trd, y_: trl, keep_prob: 0.5})
    N += 1

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0
Epoch: 0, Training Accuracy: 0.200000, Test Accuracy: 0.164000
Epoch: 59, Training Accuracy: 0.460000, Test Accuracy: 0.292000
Epoch: 119, Training Accuracy: 0.580000, Test Accuracy: 0.292000
Epoch: 178, Training Accuracy: 0.620000, Test Accuracy: 0.292000
Epoch: 238, Training Accuracy: 0.600000, Test Accuracy: 0.292000
Epoch: 297, Training Accuracy: 0.660000, Test Accuracy: 0.292000
Epoch: 357, Training Accuracy: 0.500000, Test Accuracy: 0.300000
Epoch: 416, Training Accuracy: 0.660000, Test Accuracy: 0.324000
Epoch: 476, Training Accuracy: 0.640000, Test Accuracy: 0.300000
Epoch: 535, Training Accuracy: 0.640000, Test Accuracy: 0.312000
Epoch: 595, Training Accuracy: 0.680000, Test Accuracy: 0.320000
Epoch: 654, Training Accuracy: 0.560000, Test Accuracy: 0.320000
Epoch: 714, Training Accuracy: 0.640000, Test Accuracy: 0.324000
Epoch: 773, Training Accuracy: 0.800000, Test Accuracy: 0.332000
Epoch: 833, Train

KeyboardInterrupt: 

In [48]:
if not os.path.isfile('glove.6B.zip'):
    urllib.request.urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
glove = {}
with zipfile.ZipFile('glove.6B.zip', 'r') as z:
    with z.open('glove.6B.50d.txt', 'r') as file:
        for line in file:
            tokens = line.decode().replace("\r","").replace("\n","").split()
            if len(tokens) < 2: continue
            word = tokens[0]
            vec = np.array(list([float(i) for i in tokens[1:] if len(i) > 1]))
            if vec.shape[0] < 50: continue
            glove[word] = vec

In [51]:
train_bow_glove = []

for talk in train_data_expanded:
    bow = np.zeros(50)
    count = 0
    for sent in talk:
        for word in sent:
            if word in glove:
                bow += glove[word]
                count += 1
    bow = bow / count
    train_bow_glove.append(bow)
    
print(len(train_bow_glove))

8440


In [52]:
test_bow_glove = []

for talk in test_data:
    bow = np.zeros(50)
    count = 0
    for sent in talk:
        for word in sent:
            if word in glove:
                bow += glove[word]
                count += 1
    bow = bow / count
    test_bow_glove.append(bow)
    
print(len(test_bow_glove))

250


In [54]:
embedding_size = 50
node_count_1 = 100
node_count_2 = 100

x = tf.placeholder(tf.float32, shape=[None, embedding_size])
y_ = tf.placeholder(tf.float32, shape=[None, 8])

# Define the first layer here
W1 = weight_variable([embedding_size, node_count_1])
b1 = bias_variable([node_count_1])
h1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

# Use dropout for this layer (should you wish)
keep_prob = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h1, keep_prob)

# Define the second layer here
W2 = weight_variable([node_count_1, node_count_2])
b2 = bias_variable([node_count_2])
h2 = tf.nn.sigmoid(tf.matmul(h_drop, W2) + b2)

# Define the output layer here
V = weight_variable([node_count_2, 8])
c = bias_variable([8])
y = tf.nn.softmax(tf.matmul(h2, V) + c)

# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cross_entropy)

# Start a tf session and run the optimisation algorithm
sess = tf.Session()
sess.run(tf.initialize_all_variables())

training_glove = Iterator(train_bow_glove, train_labels_onehot_expanded)
testing_glove = Iterator(test_bow_glove, test_labels_onehot)
train_accuracy = 0
test_accuracy = 0
N = 0

print(training.epochs)

while training_glove.epochs < 10000:
    trd, trl = training_glove.next_batch(50)
    if N % 10000 == 0:
        ted, tel = testing_glove.next_batch(250)
        train_accuracy = sess.run(accuracy, feed_dict={x: trd, y_: trl, keep_prob: 1.0})
        test_accuracy = sess.run(accuracy, feed_dict={x: ted, y_: tel, keep_prob: 1.0})
        print("Epoch: %d, Training Accuracy: %f, Test Accuracy: %f" % (training_glove.epochs, train_accuracy, test_accuracy))
    sess.run(train_step, feed_dict={x: trd, y_: trl, keep_prob: 0.5})
    N += 1

Instructions for updating:
Use `tf.global_variables_initializer` instead.
3518
Epoch: 0, Training Accuracy: 0.100000, Test Accuracy: 0.024000
Epoch: 59, Training Accuracy: 0.600000, Test Accuracy: 0.292000
Epoch: 119, Training Accuracy: 0.560000, Test Accuracy: 0.292000
Epoch: 178, Training Accuracy: 0.640000, Test Accuracy: 0.292000
Epoch: 238, Training Accuracy: 0.580000, Test Accuracy: 0.296000
Epoch: 297, Training Accuracy: 0.600000, Test Accuracy: 0.296000
Epoch: 357, Training Accuracy: 0.600000, Test Accuracy: 0.296000
Epoch: 416, Training Accuracy: 0.600000, Test Accuracy: 0.296000
Epoch: 476, Training Accuracy: 0.600000, Test Accuracy: 0.296000
Epoch: 535, Training Accuracy: 0.680000, Test Accuracy: 0.296000
Epoch: 595, Training Accuracy: 0.660000, Test Accuracy: 0.296000
Epoch: 654, Training Accuracy: 0.560000, Test Accuracy: 0.296000
Epoch: 714, Training Accuracy: 0.820000, Test Accuracy: 0.296000
Epoch: 773, Training Accuracy: 0.620000, Test Accuracy: 0.296000
Epoch: 833, Tr

KeyboardInterrupt: 